In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from scipy.stats import t


In [2]:
df = pd.read_csv("dummy_proj.csv")

In [3]:
df.columns

Index(['Coughing', 'Sneezing', 'Body Pain', 'Headache', 'Hospital Count'], dtype='object')

In [4]:
df.values

array([[ 12,  34,  41,  11, 230],
       [  1,  33,  23,  34, 400],
       [ 65,  80,  62,  18,  21],
       [ 21, 143, 122,  90, 500],
       [ 32,  34,  41,  22, 230],
       [  1,  33,   8,  34,  12],
       [ 67,  80,  62,  18,  21],
       [ 21, 143, 122, 265,  67]], dtype=int64)

In [5]:
X_val = df.drop("Hospital Count", axis=1).values
X_val = (X_val - np.mean(X_val, axis=0)) / np.std(X_val, axis=0)

y_val = df['Hospital Count'].values;
y_val = np.expand_dims(y_val, axis=1) 

In [6]:
n_features = X_val.shape[1]

In [7]:
X_in = tf.placeholder(tf.float32, [None, n_features], "X_in")
w = tf.Variable(tf.random_normal([n_features, 1]), name="w")
b = tf.Variable(tf.constant(0.1, shape=[]), name="b")
h = tf.add(tf.matmul(X_in, w), b)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
y_in = tf.placeholder(tf.float32, [None, 1], "y_in")
loss_op = tf.reduce_mean(tf.square(tf.subtract(y_in, h)),
                         name="loss")
train_op = tf.train.GradientDescentOptimizer(0.3).minimize(loss_op)

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
#     hh = sess.run(h , feed_dict={
#             X_in: X_val})
    for batch in range(100):
        sess.run(train_op, feed_dict={
            X_in: X_val,
            y_in: y_val
        })
    w_computed = sess.run(w)
    b_computed = sess.run(b)

In [10]:
w_computed

array([[-165.69931],
       [ -84.08761],
       [ 292.59146],
       [-188.4843 ]], dtype=float32)

In [11]:
b_computed

185.125

In [12]:
X_in = tf.placeholder(tf.float32, [None, n_features], "X_in")
w_f = tf.constant(w_computed, shape = w_computed.shape)
b_f = tf.constant(b_computed,  shape = b_computed.shape)
op_preds = tf.add(tf.matmul(X_in, w_f), b_f)
errors = tf.subtract(op_preds, y_in)
RSS = tf.tensordot(tf.transpose(errors), errors, 1)

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    rss = sess.run(RSS , feed_dict={
        X_in: X_val,
        y_in : y_val})
#     sess.run(rss)
    

In [14]:
w_computed

array([[-165.69931],
       [ -84.08761],
       [ 292.59146],
       [-188.4843 ]], dtype=float32)

In [15]:
rss

array([[55575.484]], dtype=float32)

In [16]:
df = len(X_val) - n_features

In [17]:
s_sq = rss / df

In [18]:
for i in range(n_features) :
    cc = X_val[:, i]
    temp = (cc - np.mean(cc, axis=0))
    den = np.dot(temp.T, temp)
    t_for_i = w_computed[i] / (np.sqrt(s_sq / den))
#     print(t_for_word_freqc)
    p = t.sf(t_for_i, df)
    print(p)


[[0.99177249]]
[[0.94310485]]
[[0.00108386]]
[[0.99468226]]
